<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>

<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/The-Thematics/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# The Thematic map

Last session we created a file with several layers:

In [12]:
import geopandas as gpd

linkGit='https://raw.githubusercontent.com/meade68/DACSS_690D_Week03/refs/heads/main/'
linkWorldMaps='worldMaps.gpkg'

We can easily find out the layers in that geopackage file (**worldMaps.gpkg**):

In [13]:
gpd.list_layers(linkGit+linkWorldMaps)

,name,geometry_type
0,rivers,MultiLineString
1,cities,Point
2,countries,MultiPolygon


For the thematics we will retrieve only one layer (map):

In [14]:
countries=gpd.read_file(linkGit+linkWorldMaps,layer='countries')

# see some
countries

,COUNTRY,geometry
0,Aruba (Netherlands),"MULTIPOLYGON (((-69.88223 12.41111, -69.94695 ..."
1,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.75195 ..."
2,Afghanistan,"MULTIPOLYGON (((61.27656 35.60725, 61.29638 35..."
3,Algeria,"MULTIPOLYGON (((-5.15213 30.18047, -5.13917 30..."
4,Azerbaijan,"MULTIPOLYGON (((46.54037 38.87559, 46.49554 38..."
...,...,...
247,South Sudan,"MULTIPOLYGON (((34.21807 9.96458, 34.20722 9.9..."
248,Indonesia,"MULTIPOLYGON (((123.21846 -10.80917, 123.19832..."
249,East Timor,"MULTIPOLYGON (((124.41824 -9.3001, 124.40446 -..."
250,Curacao (Netherlands),"MULTIPOLYGON (((-68.96556 12.19889, -68.91196 ..."


As you see, the GDF above has just two colums; enough to plot a map, but no more than that.

Let me open a DF:

In [ ]:
import pandas as pd

someDataLink='WORLD/some_dataworld.csv'

someData=pd.read_csv(linkGit+someDataLink)

## data available
someData.info()

## Pre Processing: Merging

The DF has some interesting numerical data (_float64_):
* fragility: fragility index 2023 -[details here](https://fragilestatesindex.org/2023/06/14/fragile-states-index-2023-annual-report/)
* co2: metric tonnes of CO2 emmitted -[details here](https://www.cia.gov/the-world-factbook/field/carbon-dioxide-emissions/country-comparison/)
* sq_km: country area -[details here](https://www.cia.gov/the-world-factbook/field/area/country-comparison/)
* num_airports: count of airports in the country -[details here](https://www.cia.gov/the-world-factbook/field/airports/country-comparison/)
* population: country population  -[details here](https://www.cia.gov/the-world-factbook/field/population/country-comparison/)
* mobiles: total number of mobile cellular telephone subscribers -[details here](https://www.cia.gov/the-world-factbook/field/telephones-mobile-cellular/country-comparison/)

There are also other columns that may be of help:

In [ ]:
someData.head()

Preparing thematic maps requires **social data** about the geometry (line, polygon, point). The "countries" geoDF has no social data, so the preprocessing requires merging the DF into the GDF.

Merging is not a trivial process. For this case, it will even require fuzzy merging. You may see the full mergin process in this [GoogleColab notebook](https://colab.research.google.com/drive/1iGTr8z1Bo8sitgg7uNcKh33pwleymb8O?usp=sharing).

We will use the file produced by that colab notebook

In [ ]:
linkToIndicators="https://github.com/DACSS-Spatial/data_forSpatial/raw/refs/heads/main/WORLD/worldindicators.json"
theMapAndData=gpd.read_file(linkToIndicators)
theMapAndData.info()

In [ ]:
theMapAndData

As you see, the columns from the DF (someData) are now part of the GDF (countries), that is now in this new GDF **theMapAndData**.

# Thematics: the DDM (Dot Density Map)

You have a DDM if you use **dots** to show comparatively which polygon of a map has 'more' (or 'less') of some countable phenomenon of interest.
A dot is an aggregated value, but it is constant for each dot.

We have the world map, let's see the regions we have:

In [ ]:
theMapAndData.region.value_counts()

### Preprocessing: Filtering

Let's keep Africa for this session:

In [ ]:
Africa=theMapAndData[theMapAndData.region=='AFRICA']
Africa.plot()

**What variable to use?** Let's go with population (countable).

A DDM can serve as an effective way to show the distribution of people. The most critical tasks are:

* Unit per Dot: Deciding how many people each dot should represent is key for visual clarity. This value should be chosen carefully to avoid oversaturation or sparsity.

* Dot Placement: The dots should be randomly distributed within each polygon to avoid misrepresenting the data.

Every dot represents the same amount, then a DDM uses raw counts (non-normalized). The size of all dots on the map are the same, which  ensures a consistent visual comparison.

### Preprocessing: NEW UNIT

How many people will the dot represent? There is some **trial and error** to make this decision, but it helps exploring the variable descriptives.

In [ ]:
print(Africa.population.describe().to_string(float_format=lambda x: f'{x:.0f}'))

From what we see, we should not choose 100 000 as the unit per dot. At least one country will have zero dots. Then, the trial and error is about choosing values like 1000, 10000, 20000, etc., but below 98187.

If we chose 25000, this would be the amount of points within each polygon (country):

In [ ]:
(Africa.set_index("Country").population / 25000).astype(int).sort_values()

This is a safe option (no zeroes).

### Preprocessing: NEW LAYER

We need to create a map of dots, a new layer. The source will be the Africa polygons we currently have. Let's do that first:

In [ ]:
# a copy of Africa
temporal_gdf=Africa.copy()
temporal_gdf.head()

the GeoDF **temporal_gdf** is just an independent copy of Africa, as you see all the geometries are MULTIPOLYGONS still. Let me replace the row names by the country name:

In [ ]:
# the row names will be the country
temporal_gdf.set_index('Country',inplace=True)

# we have this now
temporal_gdf.head()

Let's create **dots_per_country**, the column that represents the amount of dots for each country (like we saw before):

In [ ]:
# creating  column 
temporal_gdf['dots_per_country'] = (Africa['population'] / 25000).astype(int).values # notice .values!

# see
temporal_gdf['dots_per_country'].head(10)

Let's find a **random_location** for the dots:

In [ ]:
# enter 'sample_points()'
temporal_gdf.sample_points(size=temporal_gdf['dots_per_country'],rng=123).head(10)

That was a critical step, you created a...

In [ ]:
type(temporal_gdf.sample_points(size=temporal_gdf['dots_per_country'],rng=123))

Take a look at one of this series' element:

In [ ]:
# this is the case of Comoros:
temporal_gdf.sample_points(size=temporal_gdf['dots_per_country'],rng=123).loc['COMOROS']

As usual, that GeoS is turned into a GeoDF:

In [ ]:
Africa_dots=gpd.GeoDataFrame(geometry=temporal_gdf.sample_points(size=temporal_gdf['dots_per_country'],rng=123))
Africa_dots.head(10)

Now you have a GDF:

In [ ]:
Africa_dots.info()

Here is our DDM:

In [ ]:
base=Africa.plot(facecolor="white",#color of polygon fill
               edgecolor='grey') #color of border
Africa_dots.plot(markersize=0.02, color='red',ax=base, alpha=0.2)

It is generally recommended that DDMs use an **equal-area projection**, which benefits density and limits area distortion.

In [ ]:
Africa_8857=Africa.to_crs(8857)
Africa_dots_8857=Africa_dots.to_crs(8857)

In [ ]:
# new version
base=Africa_8857.plot(facecolor="white",
                              edgecolor='grey')
Africa_dots_8857.plot(markersize=0.02, color='red',ax=base, alpha=0.2)

### Exporting to R?

Africa_dots_8857 has a column of multi-points as we know. Shapefiles can not deal with this structure not visual programs in R such as **leaflet**. Then, we need to split those multipoints using **explode()**:

In [ ]:
Africa_dots_8857.explode()

We may just need to get rid of the indexes:

In [ ]:
Africa_dots_8857.explode(ignore_index=True)

The function **explodes()** returns...

In [ ]:
type(Africa_dots_8857.explode(ignore_index=True))

This is a safe option outside Python:

In [ ]:
# then
Africa_dots_8857=Africa_dots_8857.explode(ignore_index=True)

# Thematics: the PSM (Proportional Symbol Map)

You have a PSM if you use a symbol (generally a circle) to show the distribution of a variable per location.
The symbol is an aggregated raw value, and its size varies according to those values. So now we need:

### Preprocessing: scale the variable to determine size

How to adjust the size of a symbols based of population? We should not have symbol size five million!

In [ ]:
##what about square root scaling
import numpy as np

max_size=40
min_size=1
scaled_sizes = np.sqrt(
    (Africa_8857.population - Africa_8857.population.min()) / (Africa_8857.population.max() - Africa_8857.population.min())
) * (max_size-1) + min_size

scaled_sizes.sort_values()

### Preprocessing: create new layer

Similar to DDM, the PSM is a new layer, built from the original polygons:

In [ ]:
Africa_8857_symbols = Africa_8857.copy()
Africa_8857_symbols.set_index('Country',inplace=True)

#this is just a copy (new index names)
Africa_8857_symbols.head()

Let's set the **size** of the symbol:

In [ ]:
Africa_8857_symbols['size']=scaled_sizes.values # notice .values!

### Preprocessing: location for the symbol

We need one symbol per polygon, an obvious choice is the centroid.

In [ ]:
# Africa_8857_locations will have a new geometry:
Africa_8857_symbols['geometry'] = Africa_8857_symbols['geometry'].centroid

# see the geom is NOW a POINT
Africa_8857_symbols

We got the basics, then:

In [ ]:
base=Africa_8857.plot(facecolor="white",
                      edgecolor='grey')
# Plot the centroids on top
Africa_8857_symbols.plot(
    ax=base,
    markersize=Africa_8857_symbols['size'],
    color='black'
)

Keep in mind that the standard way of computing centroids may bring some trouble if you have multipolygons (archipelagos?). Let´s plot Seychelles:

In [ ]:
base=Africa_8857[Africa_8857.Country=="SEYCHELLES"].explore()
Africa_8857_symbols[Africa_8857_symbols.index=="SEYCHELLES"].explore(m=base,color="red")

The location is nowhere within a polygon. An  alternative is **representative_point()**:

In [ ]:
# replace the points
Africa_8857_symbols["geometry"]=Africa_8857.representative_point().values

We will keep that last result.

In [ ]:
base=Africa_8857[Africa_8857.Country=="SEYCHELLES"].explore()
Africa_8857_symbols[Africa_8857_symbols.index=="SEYCHELLES"].explore(m=base,color="red")

### Preprocessing: detect outliers

PPMs are not thematic maps to see precise values, but to reveal clear differences on the variable distribution used. As human eyes are very limited to detect area differences, we may need to use some other tactics to confirm if we have outliers:

In [ ]:
boxplotInfo=Africa_8857_symbols.boxplot(column='population',return_type="dict")

The object **boxplotInfo** showed the boxplot, and we confirm there are outliers. We can recover them like this:

In [ ]:
outliers=boxplotInfo['fliers'][0].get_ydata()
## see
outliers

Then, these are the outlying countries:

In [ ]:
Africa_8857_symbols[Africa_8857_symbols.population.isin(outliers)].index

Knowing this information, we can create a column as outlier flag:

In [ ]:
Africa_8857_symbols['population_outlier']=Africa_8857_symbols.population.isin(outliers)*1

## see
Africa_8857_symbols

We can use that like this:

In [ ]:
base = Africa_8857.plot(color='white', edgecolor='black', figsize=(10,10))

# Define your color map
mapcolor = {1: 'red', 0: 'grey'}

# Plot the centroids on top
Africa_8857_symbols.plot(
    ax=base,
    markersize=Africa_8857_symbols['size'],
    color=Africa_8857_symbols['population_outlier'].map(mapcolor)
)

## Exporting to R?

It is **almost** ready. R will use the new layer and the location of the points. However, R's ggplot needs NOT your size variable. Give GGPLOT the _population_ variables and use **scale_size_area()** to manage the size and the legend.

Also, notice that I have the country names as index, and the other columns might be redundant (they are in **Africa_8857**):

In [ ]:
Africa_8857_symbols.head()

In [ ]:
## then
Africa_8857_symbols.reset_index(drop=False, inplace=True)
Africa_8857_symbols.drop(columns=['iso2', 'region', 'fragility', 'co2', 'sq_km', 'num_airports','mobiles'], inplace=True)

## final for R:
Africa_8857_symbols.head()

Done!

# Thematics: Choropleths

Notice that we do not normalized DDMs nor PSMs: data is just a count or a representation of counts. Choropleths will 'paint' the whole polygon and as noticed in PSMs, we have to be very careful when using color in shape, as the area itself is a confounding. To control this visual artifact, the variable must be normalized, that is, divided by a value correlated with the area.

Then, for this course, a choropleth is a normalized representation of data.

Following our definition:

* This is **NOT** a choropleth:


In [ ]:
Africa_8857.plot(Africa_8857.population,cmap="YlOrRd")

### Preprocessing: Normalization

* This **IS** a choropleth:

In [ ]:
Africa_8857.plot(Africa_8857.population/Africa_8857.sq_km,cmap="YlOrRd")

Choropleths are great to represent normalized indicators such as:
* Densities
* Proportion
* Ratios
* Averages

This is a density:

In [ ]:
Africa_8857['population_density']=Africa_8857.population/Africa_8857.sq_km

### Preprocessing: Discretizing

Population by area is a density indicator. This is a real number (not an integer), so most of the time the color gradient might  not be that useful to reveal some pattern visually. Then, we often discretize (or bin) the indicator computed.
Now, we want to cut the variable. Run the next code to make sure you have tose packages:

In [ ]:
## do you have these installed?
# ! pip show numba mapclassify numpy

Let's explore the bining algorithms:

In [ ]:
import mapclassify
import numpy as np

np.random.seed(12345) # so we all get the same results!

# let's try 5 intervals
K=5
theVar=Africa_8857['population_density']

# same interval width, easy interpretation
ei5 = mapclassify.EqualInterval(theVar, k=K)
# same interval width based on standard deviation, easy - but not as the previous one, poor when high skewness
msd = mapclassify.StdMean(theVar)
# interval width varies, counts per interval are close, not easy to grasp, repeated values complicate cuts
q5=mapclassify.Quantiles(theVar,k=K)

# based on similarity, good for multimodal data
mb5 = mapclassify.MaximumBreaks(theVar, k=K)
# based on similarity, good for skewed data
ht = mapclassify.HeadTailBreaks(theVar) # no K needed
# based on similarity, optimizer
fj5 = mapclassify.FisherJenks(theVar, k=K)
# based on similarity, optimizer
jc5 = mapclassify.JenksCaspall(theVar, k=K)
# based on similarity, optimizer
mp5 = mapclassify.MaxP(theVar, k=K)

How can we select the right classification?
Let me use the the Absolute deviation around class median (ADCM) to make the comparisson:

In [ ]:
class5 = ei5,msd, q5,mb5,  ht, fj5, jc5, mp5
# Collect ADCM for each classifier
fits = np.array([ c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms['classifier'] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ['ADCM', 'Classifier']

Now, plot the **adcms**:

In [ ]:
adcms.sort_values('ADCM').plot.barh(x='Classifier')

Let's save the best strategy:

In [ ]:
Africa_8857['population_density_FJ5'] = fj5.yb # yb will give you a numeric label

In [ ]:
# there you are
Africa_8857[['population_density','population_density_FJ5']].head(20)

We could create a copy of that column to add descriptive labels:

In [ ]:
# renaming
newLabelsForLevels={0:"0_VeryLow", 1:"1_Low", 2:"2_Middle", 3:"3_High", 4:"4_VeryHigh"}

Africa_8857['population_density_FJ5_cat']=Africa_8857.loc[:,'population_density_FJ5'].replace(newLabelsForLevels)

# we have
Africa_8857[['population_density','population_density_FJ5','population_density_FJ5_cat']].head(20)

We are ready for a discrete choropleth:

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, figsize=(10, 10))
Africa_8857.plot(column='population_density_FJ5_cat', # variable to plot
                 cmap='viridis_r', # set of colors
                 categorical=True, # can be interpreted as category
                 edgecolor='grey', # border color
                 linewidth=0.3, # width of border
                 legend=True, # need a legend?
                 ax=ax
       )

ax.set_axis_off()

# Saving

Let's save these maps for R.

In [ ]:
## Good practice to refresh your memory

## Find only GeoDataFrames in the current session
geodataframe_list = [var for var in globals() if isinstance(globals()[var], gpd.GeoDataFrame)]

print("List of GeoDataFrames in memory:")
for name in geodataframe_list:
    print(name)

These are the ones needed in R:

In [ ]:
# for choropleth and base map
Africa_8857.info()

In [ ]:
# for DDMs
Africa_dots_8857.info()

In [ ]:
# for PSMs
Africa_8857_symbols.info()

The file **africa_8857.gpkg** will keep all those maps as layers!

In [ ]:
Africa_8857.to_file("africa_8857.gpkg",driver='GPKG',layer='continent')
Africa_dots_8857.to_file("africa_8857.gpkg",driver='GPKG',layer='population_ddm')
Africa_8857_symbols.to_file("africa_8857.gpkg",driver='GPKG',layer='population_psm')

Take a look at the [Dashboard](https://dacss-spatial.github.io/The-Thematics/draft_storyboard_tabs.html)